In [14]:
import numpy as np
import pandas as pd
import sys


class GSEA:
    def __init(self):
        self.df=None
        self.samp=None
        self.expr_gene=None
        self.pathway_gene_dict = None
        self.largest=None
        self.nt=None
        
    def load_expression_data(self,file_name,drop):
        '''
        input: gene expression filename
        output: pandas dataframe
        '''
        df = pd.read_csv(file_name,sep='\t')
        if drop:
            df = df.drop(columns='SYMBOL')
        #print(df.head())
        return df

    def get_expression_genes(self,df):
        '''
        input: dataframe
        output: list of genes, 10712
        '''
        df_small = df.iloc[:,df.columns=="SYMBOL"]
        #print(df_small.shape)
        #print(df_small.head())
        gene_list = df_small.to_numpy().flatten()
        #print("len expression genes:",len(gene_list),gene_list.shape)
        return gene_list 

    def load_samples(self,file_name):
        '''
        input: filename of samples
        output: distionary, key=patient name, value=0 healthy, 1 diseased
        '''
        d = {}
        with open(file_name, "r") as fh:
            lines = fh.readlines()
            for line in lines:
                tokens = line.split()
                d[tokens[0]] = tokens[1]
        #print("d:",d)
        return d
    
    def load_kegg(self,filename):
        '''
        input:kegg file name
        output: dictionary key=KEGG_CITRATE_CYCLE_TCA_CYCLE value=list of genes which are in expression data
        all_genes all genes in all pathways no screening
        hum_genes, only count ones in expression file
        glist only count ones in expression file. glist/path_gene and hum_genes difference? pathgene dict of pathway, list of genes in expressoin
        '''
        path_gene = {}
        with open(filename, "r") as fh:
            lines = fh.readlines()
            print("len(lines):",len(lines))
            for i in range(0,len(lines)):
                tokens = lines[i].split()
                #print("pathway:",tokens[0])
                genes = tokens[2:]
                if i==0:
                    print("lines[i]:",lines[i])
                    print("tokens[2:]",tokens[2:],len(tokens[2:]))
                print("pathway genes:",genes,"num pathway genes:",len(genes))
                g_list=[]
                for g in genes:
                    print("+++++++++++testing g:",g)
                    if g in self.expr_gene:
                        print("g is in self.expr_gene!!!!!")
                        g_list.append(g)
                        #if gene in expression data add to hum_gene dict
                if i==0:
                    print("len g_list, should match later query:",len(g_list))
                path_gene[tokens[0]] = g_list
                #path_gene[tokens[0]] =tokens[2:]
                #print("pathway:",tokens[0]," num genes:",len(g_list))
            print(" len path_gene:",len(path_gene))
            return path_gene

    
    def load_data(self,expfile,sampfile,keggfile):
        self.samp = self.load_samples(sampfile)
        self.df = self.load_expression_data(expfile,False)       
        self.expr_gene = self.get_expression_genes(self.df)
        print("len expr_gene should be 10k:",len(self.expr_gene))
        print("expr_gene:",self.expr_gene)
        self.pathway_gene_dict = self.load_kegg(keggfile)
        print("************")
        print(self.pathway_gene_dict["KEGG_CITRATE_CYCLE_TCA_CYCLE"])
        print(len(self.pathway_gene_dict["KEGG_CITRATE_CYCLE_TCA_CYCLE"]))
        print("************")
        #pathway_gene_dict = load_kegg("c2.cp.kegg.v6.2.symbols.filtered.gmt",expr_gene)
        

    def get_gene_rank_order(self):
        print("------df head--------")
        print(self.df.head())
        print("---------------------")
        rows,cols = self.df.shape
        #build healthy and diseased indexes
        print(self.df.columns)
        healthy_idx=[]
        disease_idx=[]
        for idx,col in enumerate(self.df.columns):
            if col!="SYMBOL":
                print("idx:",idx,"col:",col,self.samp[col])
                if self.samp[col]=='1':
                    disease_idx.append(idx)
                elif self.samp[col]=='0':
                    healthy_idx.append(idx)
                else:
                    print("xxxx")

        print("healthy_idx:",healthy_idx," disease_idx:",disease_idx)
        df_np = self.df.to_numpy()
        print("rows:",rows,"cols:",cols)#fetch healthy as list of columns, diseased as list of columns
        rank_me=[]
        healthy_mean=np.mean(df_np[0,[healthy_idx]])
        disease_mean=np.mean(df_np[0,[disease_idx]])
        for idx in range(0,df_np.shape[0]):
            #print("df iloc:",df.iloc[idx,0],df_np[0,[healthy_idx]],healthy_mean)
            #print(df_np[0,[disease_idx]],disease_mean)
            #print("diff:",disease_mean-healthy_mean)
            healthy_mean=np.mean(df_np[idx,[healthy_idx]])
            disease_mean=np.mean(df_np[idx,[disease_idx]])
            rank_me.append((idx,self.df.iloc[idx,0],disease_mean-healthy_mean)) #sort by second tuple value

        print("length ranked genes: ",len(rank_me))
        #before sorting
        print("rank_me[0:5]",rank_me[0:5])
        largest_first = sorted(rank_me, key = lambda x: x[2],reverse=True)
        print("largest_first[0:5]",largest_first[0:5],len(largest_first))
        #return list of genes only largest first
        return_me=[]
        for x in largest_first:
            return_me.append(x[1])
        print("return_me[0:5]:",return_me[0:5],len(return_me))
        self.nt=len(return_me)
        self.largest = return_me
        return return_me
    
    def get_enrichment_score(self,geneset):
        print("num pathways, should be 145:",len(self.pathway_gene_dict))
        print("pathway genes",self.pathway_gene_dict[geneset],len(self.pathway_gene_dict[geneset]))
        
        ng = len(self.pathway_gene_dict[geneset])
        nt=self.nt
        penalty_add =np.sqrt((nt-ng)/ng)
        penalty_miss = np.sqrt(ng/(nt-ng))
        #print("ng:",ng," penalty_add:",penalty_add," penalty_miss:",penalty_miss)    
        #calculate ES for each geneset pathway_gene_dict[x] using ranked list. wow for all 10k? 
        es=[]
        es.append(0.)
        sum_es=0.
        for idx in range(0,len(self.largest)):
            #print(idx,"processing gene:",self.largest[idx])
            #this is right list? not largest first??? wait the geneset has to be in expressoin data
            if self.largest[idx] in self.pathway_gene_dict[geneset]:
                sum_es +=penalty_add
            else:
                sum_es-=penalty_miss
            es.append(sum_es)
        #print("es:",es)
        print("max:",max(es))
        
        
        
        
        
        
        
gsea=GSEA()
gsea.load_data("GSE25628_filtered_expression.txt","GSE25628_samples.txt","test.gmt")
#this mirrors self.largest set in class
l = gsea.get_gene_rank_order()
gsea.get_enrichment_score("KEGG_CITRATE_CYCLE_TCA_CYCLE")

len expr_gene should be 10k: 10702
expr_gene: ['A2M' 'A4GALT' 'AAAS' ... 'ZYX' 'ZZEF1' 'ZZZ3']
len(lines): 1
lines[i]: KEGG_CITRATE_CYCLE_TCA_CYCLE	http://www.broadinstitute.org/gsea/msigdb/cards/KEGG_CITRATE_CYCLE_TCA_CYCLE	IDH3B	DLST	PCK2	CS	PDHB	PCK1	PDHA1	LOC642502	PDHA2	LOC283398	FH	SDHD	OGDH	SDHB	IDH3A	SDHC	IDH2	IDH1	ACO1	ACLY	MDH2	DLD	MDH1	DLAT	OGDHL	PC	SDHA	SUCLG1	SUCLA2	SUCLG2	IDH3G	ACO2

tokens[2:] ['IDH3B', 'DLST', 'PCK2', 'CS', 'PDHB', 'PCK1', 'PDHA1', 'LOC642502', 'PDHA2', 'LOC283398', 'FH', 'SDHD', 'OGDH', 'SDHB', 'IDH3A', 'SDHC', 'IDH2', 'IDH1', 'ACO1', 'ACLY', 'MDH2', 'DLD', 'MDH1', 'DLAT', 'OGDHL', 'PC', 'SDHA', 'SUCLG1', 'SUCLA2', 'SUCLG2', 'IDH3G', 'ACO2'] 32
pathway genes: ['IDH3B', 'DLST', 'PCK2', 'CS', 'PDHB', 'PCK1', 'PDHA1', 'LOC642502', 'PDHA2', 'LOC283398', 'FH', 'SDHD', 'OGDH', 'SDHB', 'IDH3A', 'SDHC', 'IDH2', 'IDH1', 'ACO1', 'ACLY', 'MDH2', 'DLD', 'MDH1', 'DLAT', 'OGDHL', 'PC', 'SDHA', 'SUCLG1', 'SUCLA2', 'SUCLG2', 'IDH3G', 'ACO2'] num pathway genes: 32
++++

penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss

penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss

penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty add
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss


penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss
penalty miss

In [ ]:
f